In [79]:
import xarray as xr
import os
import numpy as np
import pandas as pd

In [63]:
res = 2.8125
local_dir = f"E:\\weatherbench\\{res}deg"

In [69]:
d=xr.open_dataset("E:\\weatherbench\\2.8125deg\\toa_incident_solar_radiation\\toa_incident_solar_radiation_2.8125deg\\toa_incident_solar_radiation_1979_2.8125deg.nc")

In [94]:
lon = 0.0
lat = 
#data = d.query(f"lon=={lon} and lat=={lat}")
data = d.sel(lon=lon, lat=lat).to_dataframe()['tisr']
one_year_daily=np.array([data[i:i+24].mean(axis=0) for i in range(0, len(data), 24)])

In [101]:
def get_all_4_corners():
    d = xr.open_dataset("E:\\weatherbench\\2.8125deg\\2m_temperature\\2m_temperature_2.8125deg\\2m_temperature_1979_2.8125deg.nc")
    lats = [lt.item() for lt in d['lat']]
    lons = [ln.item() for ln in d['lon']]
    output = []
    for i, lat in enumerate(lats):
        for j, lon in enumerate(lons):
            if i == len(lats)-1:
                continue
            if j == len(lons)-1:
                continue
            output.append([(lat, lon), (lat, lon[i+1]), (lat[i+1], lon), (lat[i+1], lon[i+1])])
    return output

In [99]:
def get_all_4_corners():
    d = xr.open_dataset("E:\\weatherbench\\2.8125deg\\2m_temperature\\2m_temperature_2.8125deg\\2m_temperature_1979_2.8125deg.nc")
    lats = [lt.item() for lt in d['lat']]
    lons = [ln.item() for ln in d['lon']]
    output = []
    for i, lat in enumerate(lats):
        for j, lon in enumerate(lons):
            if i == len(lats)-1:
                continue
            if j == len(lons)-1:
                continue
            output.append([(lat, lon), (lat, lon[i+1]), (lat[i+1], lon), (lat[i+1], lon[i+1])])
    return output


def get_daily_value_for_year(feature_info, lat, lon, year):
    feature, df_fld = feature_info
    d = xr.open_dataset(os.path.join(local_dir, feature, f"{feature}_{res}deg", f"{feature}_{year}_{res}deg.nc"))
    data = d.sel(lon=lon, lat=lat).to_dataframe()[df_fld]
    # hourly temperature: each day covers 24 entries. Take the daily average over 24hrs
    one_year_daily=np.array([np.mean(data[i:i+24]) for i in range(0, len(data), 24)])
    return one_year_daily[:365].reshape((1, 365))


def linearize(x):
    linearized = np.array([])
    for chunk in x:
        linearized = np.concatenate((linearized, chunk), axis=0)
    return linearized


def detrend(feature, lat, lon, years, nombor):
    """
    Extract the mean feature value for each day in a year over the period between y1 and y2 inclusive and use
    it to do seasonal detrending.
    """
    y1, y2 = years
    data = None
    for y in range(y1, y2+1):
        more_data = get_daily_value_for_year(feature, lat, lon, y)
        if data is None:
            data = more_data
        else:
            data = np.concatenate((data, more_data), axis=0)
#    np.savetxt(f"daily_averages{nombor}.txt", data, delimiter=',')
    # means is an array where mean[0] is the feature value average over the years between y1 and y2 on the 
    # on the first day of a year
    means = np.array([np.array([data[i, j] for i in range(y2-y1+1)]).mean() for j in range(365)])
    detrended = data - means
    return detrended

In [100]:
get_daily_value_for_year(("toa_incident_solar_radiation", 'tisr'), 1.40625, 0.0, 1979)

array([[1460948.125, 1461980.375, 1463067.125, 1464228.   , 1465472.625,
        1466774.625, 1468144.5  , 1469581.875, 1471103.   , 1472639.875,
        1474247.875, 1475913.375, 1477630.5  , 1479399.125, 1481232.125,
        1483093.625, 1485007.   , 1486955.625, 1488949.375, 1490981.5  ,
        1493017.   , 1495071.875, 1497184.375, 1499326.   , 1501474.   ,
        1503605.875, 1505792.125, 1507963.   , 1510155.875, 1512381.   ,
        1514603.   , 1516789.625, 1518998.625, 1521188.5  , 1523362.375,
        1525510.375, 1527651.625, 1529783.625, 1531896.375, 1533980.125,
        1536009.   , 1538031.625, 1540025.   , 1541977.375, 1543877.375,
        1545732.875, 1547572.125, 1549366.5  , 1551122.375, 1552800.625,
        1554440.5  , 1556003.375, 1557527.625, 1558993.875, 1560399.   ,
        1561759.125, 1563020.   , 1564248.125, 1565405.875, 1566479.5  ,
        1567473.125, 1568399.5  , 1569286.625, 1570071.375, 1570798.125,
        1571434.875, 1572007.375, 1572508.875, 1572